In [1]:
# uncomment and run if mismo is not installed
# %pip install -q git+https://github.com/NickCrews/mismo@main

In [2]:
from __future__ import annotations

import ibis
from ibis import _
from ibis.expr import types as ir

import mismo

ibis.options.interactive = True

We are going to dedupe the PATSTAT dataset. Every record represents a patent filed,
and our task is to determine which one came from the same inventor.
The end goal is to add a column called `inventor_id` to each patent.
This dataset contains a ground truth label, so we can evaluate how well we did.

`Mismo` contains this as an included dataset so it is easy to get started.
The returned dataset is an Ibis table, which is a lazy representation of a SQL table.
It is similar to a pandas dataframe, but has a few properties that make it much
better for the record linkage use case:

- Since it is SQL backed, it can handle datasets that are larger than memory, in
  the many millions of rows.
- Computation is performed by the powerful SQL backend of your choice: Google BigQuery,
  Apache Spark, Snowflake, etc. For this demo, we use DuckDB, which is a 
  state-of-the-art SQL engine based around a columnar data model
  (ie oriented towards the bulk operations of record linkage)
- Ibis is strongly typed, has a full API, is well-documented, and has good
  integration with the rest of the python data science ecosystem.


In [3]:
patents = mismo.datasets.load_patents()
print(patents.count())
patents

2379


┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ record_id ┃ label_true ┃ name_true            ┃ name                         ┃ latitude ┃ longitude ┃ coauthors                                                                        ┃ classes                                                          ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ int64     │ int64      │ string               │ string                       │ float64  │ float64   │ string                                                                           │ string                                                           │
├───────────┼────────────┼──────────────────────┼──────────────────────────────┼──────────┼───────────┼──────────────────────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────────┤
│      2909 │     402600 │ AGILENT TECHNOLOGIES │ * AGILENT TECHNOLOGIES, INC. │     0.00 │  0.000000 │ KONINK PHILIPS ELECTRONICS N V**DAVID E  SNYDER**THOMAS D  LYSTER                │ A61N**A61B                                                       │
│      3574 │     569309 │ AKZO NOBEL           │ * AKZO NOBEL N.V.            │     0.00 │  0.000000 │ TSJERK  HOEKSTRA**ANDRESS K  JOHNSON**TERESA MARIE  CHERON**ALBERTO  SLIKTA**JA… │ G01N**B01L**C11D**G02F**F16L                                     │
│      3575 │     569309 │ AKZO NOBEL           │ * AKZO NOBEL NV              │     0.00 │  0.000000 │ WILLIAM JOHN ERNEST  PARR**HANS  OSKARSSON**MARTIN  HELLSTEN**KORNELIS  OVERKEM… │ C09K**F17D**B01F**C23F                                           │
│      3779 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │    52.35 │  4.916667 │ GUENTER  KOCHSMEIER**ZBIGNIEW  WIEGOLASKI**EVAN JOHN  STANBURY**PETER GRANT  JE… │ G02B**G04G**H02G**G06F                                           │
│      3780 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │    52.35 │  4.916667 │ ZILAN  MANFRED**JOSIANE  RAMOS**DUANE LYNN  MORTENSEN**CHRISTIAN  LE SERGENT     │ H03G**B05D**H04L**H04B**C03B**C03C**G02B**H01B                   │
│      3782 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │     0.00 │  0.000000 │ OLIVIER  AUDOUIN**MICHEL  SOTOM**JEAN MICHEL  GABRIAGUES                         │ H04B**H01S**H04J                                                 │
│     15041 │    4333661 │ CANON EUROPA         │ * CANON EUROPA N.V           │     0.00 │  0.000000 │ LEE  RICKLER**SIMON  PARKER**CANON RES CENT EURO **RAKEFET  SAGMAN**TIMOTHY FRA… │ G06F                                                             │
│     15042 │    4333661 │ CANON EUROPA         │ * CANON EUROPA N.V.          │     0.00 │  0.000000 │ QI HE  HONG**ADAM MICHAEL  BAUMBERG**ALEXANDER RALPH  LYONS                      │ G06T**G01B                                                       │
│     15043 │    4333661 │ CANON EUROPA         │ * CANON EUROPA NV            │     0.00 │  0.000000 │ NILESH  PATHAK**MASAMICHI  MASUDA** CANON TECHNOLOGY EURO **PATRICK WILLIAM  MO… │ H04B**G06T**G06F**H04M**H04N**H04Q**G03B**B41J**G01B**G06Q       │
│     25387 │    7650783 │ DSM                  │ * DSM N.V.                   │     0.00 │  0.000000 │ GABRIEL MARINUS  MEESTERS**RUDOLF CAROLUS  BARENDSE**ARIE KARST  KIES**ALEXANDE… │ C12N**A61K**A23L**A23J**A23K**A01H**B01J**C12R**C07D**A61P**B01D │
│         … │          … │ …                    │ …                            │        … │         … │ …                                                                                │ …  

In [4]:
from mismo import clean

clean.distribution_dashboard(patents, column="name")

Let's clean this up a bit:
- clean up whitespace
- convert the `classes` column to actual arrays (they really represent sets).
  Each element in `classes` is a 4-character IPC technical code that is like a tag
  for the patent. Similar patents will have similar tags.
- convert `coauthors` into a sorted list of tokens.

Of course, this sort of cleaning only happens after a lot of exploring the
data manually. This isn't anything in particular to record linkage,
but this needs to happen for any sort of data analysis.

If you're not familiar with Ibis, this is useful to give you a sense
of what it's API is like. Mismo tries to just use Ibis's API whenever possible,
and doesn't re-invent the wheel.


In [5]:
def clean_names(names: ir.StringValue) -> ir.StringValue:
    names = clean.norm_whitespace(names)
    names = names.upper()
    # Only want to keep letters, numbers, and spaces
    names = names.re_replace("[^0-9A-Z ]", "")
    # Some company names have "NV" or "BV" in them, which is a Dutch
    # classification of businesses sort of like LLC vs Corporation in the US.
    # These seem to just be noise and don't help us distinguish between
    # different companies, so we want to remove them.
    # However, sometimes the names are ill-formed or have other junk in them
    # like "philipsN V" or "philipsN.V."
    names = names.re_replace(r"B\s*V", "")
    names = names.re_replace(r"N\s*V", "")
    # Also remove other common stopwords that usually just add noise
    names = names.re_replace(r"\bCO\b", "")
    names = names.re_replace(r"\bCORP\b", "")
    names = names.re_replace(r"\bLTD\b", "")
    names = names.re_replace(r"\bINC\b", "")
    names = names.re_replace(r"\bLLC\b", "")
    names = names.re_replace(r"\bNL\b", "")
    # Now have to do whitespace fixup again
    names = clean.norm_whitespace(names)
    return names


def clean_coauthors(s: ir.StringValue) -> ir.ArrayValue:
    """Turn a 'list' of coauthors into a sorted array of cleaned name tokens.

    The raw string looks like:
    "DAVID E  SNYDER**THOMAS D  LYSTER"
    With "**" as the separator. However, entire names are not always useful
    to differentiate between people: there are some records where a coauthor is
    listed as "ALKEMA DUURT PIETER WILLEM" and another as "ALKEMA DUURT".
    These are probably the same person, but if we compared entire names with each other
    that would be a false negative. So instead we turn this into a sorted array of
    name tokens, eg for the example above:
    ["DAVID", "E", "SNYDER", "THOMAS", "D", "LYSTER"].

    Except we also want to remove initials and other stopwords like "VAN"
    so scrub all tokens less than 3 characters long.
    """
    s = s.upper().nullif("NONE").replace("**", " ")
    s = clean.norm_whitespace(s)
    return s.split(" ").unique().sort().filter(lambda x: x.length() >= 3)


def clean_classes(s: ir.StringValue) -> ir.ArrayValue:
    return s.upper().split("**").map(clean.norm_whitespace).sort()


cleaned_names = clean_names(_.name)

featured = patents.select(
    "record_id",
    "label_true",
    "name_true",
    "name",
    name_cleaned=cleaned_names,
    name_tokens=cleaned_names.split(" ").map(clean.norm_whitespace).sort(),
    latitude=_.latitude.nullif(0),
    longitude=_.longitude.nullif(0),
    coauthors=clean_coauthors(patents.coauthors),
    classes=clean_classes(patents.classes),
)
featured

┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ record_id ┃ label_true ┃ name_true            ┃ name                         ┃ name_cleaned         ┃ name_tokens                 ┃ latitude ┃ longitude ┃ coauthors                         ┃ classes                  ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ int64     │ int64      │ string               │ string                       │ string               │ array<string>               │ float64  │ float64   │ array<string>                     │ array<string>            │
├───────────┼────────────┼──────────────────────┼──────────────────────────────┼──────────────────────┼─────────────────────────────┼──────────┼───────────┼───────────────────────────────────┼──────────────────────────┤
│      2909 │     402600 │ AGILENT TECHNOLOGIES │ * AGILENT TECHNOLOGIES, INC. │ AGILENT TECHNOLOGIES │ ['AGILENT', 'TECHNOLOGIES'] │     NULL │      NULL │ ['DAVID', 'ELECTRONICS', ... +5]  │ ['A61B', 'A61N']         │
│      3574 │     569309 │ AKZO NOBEL           │ * AKZO NOBEL N.V.            │ AKZO NOBEL           │ ['AKZO', 'NOBEL']           │     NULL │      NULL │ ['ALBERT', 'ALBERTO', ... +31]    │ ['B01L', 'C11D', ... +3] │
│      3575 │     569309 │ AKZO NOBEL           │ * AKZO NOBEL NV              │ AKZO NOBEL           │ ['AKZO', 'NOBEL']           │     NULL │      NULL │ ['CHRISTINA', 'ERNEST', ... +11]  │ ['B01F', 'C09K', ... +2] │
│      3779 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │ ALCATEL              │ ['ALCATEL']                 │    52.35 │  4.916667 │ ['EVAN', 'GRANT', ... +8]         │ ['G02B', 'G04G', ... +2] │
│      3780 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │ ALCATEL              │ ['ALCATEL']                 │    52.35 │  4.916667 │ ['CHRISTIAN', 'DUANE', ... +7]    │ ['B05D', 'C03B', ... +6] │
│      3782 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │ ALCATEL              │ ['ALCATEL']                 │     NULL │      NULL │ ['AUDOUIN', 'GABRIAGUES', ... +4] │ ['H01S', 'H04B', ... +1] │
│     15041 │    4333661 │ CANON EUROPA         │ * CANON EUROPA N.V           │ CANON EUROPA         │ ['CANON', 'EUROPA']         │     NULL │      NULL │ ['CANON', 'CENT', ... +11]        │ ['G06F']                 │
│     15042 │    4333661 │ CANON EUROPA         │ * CANON EUROPA N.V.          │ CANON EUROPA         │ ['CANON', 'EUROPA']         │     NULL │      NULL │ ['ADAM', 'ALEXANDER', ... +5]     │ ['G01B', 'G06T']         │
│     15043 │    4333661 │ CANON EUROPA         │ * CANON EUROPA NV            │ CANON EUROPA         │ ['CANON', 'EUROPA']         │     NULL │      NULL │ ['BARRY', 'BROCARD', ... +24]     │ ['B41J', 'G01B', ... +8] │
│     25387 │    7650783 │ DSM                  │ * DSM N.V.                   │ DSM                  │ ['DSM']                     │     NULL │      NULL │ ['ALEXANDER', 'ANDELA', ... +39]  │ ['A01H', 'A23J', ... +9] │
│         … │          … │ …                    │ …                            │ …                    │ …                           │        … │         … │ …                                 │ …                        │
└───────────┴────────────┴──────────────────────┴──────────────────────────────┴──────────────────────┴─────────────────────────────┴──────────┴───────────┴───────────────────────────────────┴──────────────────────────┘

OK, now it's time to block! This is where we generate comparisons between records.
If we were naive and generated all possible comparisons from N record,
you would end up with N^2 comparisons. For our small dataset of ~2000 records
we would be able to get away with this, but for datasets much larger than this
it would be infeasible.

For blocking, we specify an collection of `Blocker`s. A Blocker is just
a callable of the form `Table, Table, **kwargs -> Table` that takes in two tables
and returns a table of pairs of records for further processing.

In [6]:
from mismo import block

name_blocker = block.ConditionBlocker(
    lambda left, right, **_kwargs: left.name_cleaned[0:3] == right.name_cleaned[0:3],
    name="Name First 3",
)
name_blocker(featured, featured)

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ record_id_l ┃ record_id_r ┃ classes_l                ┃ classes_r                ┃ coauthors_l                       ┃ coauthors_r                       ┃ label_true_l ┃ label_true_r ┃ latitude_l ┃ latitude_r ┃ longitude_l ┃ longitude_r ┃ name_cleaned_l             ┃ name_cleaned_r                             ┃ name_l                        ┃ name_r                                              ┃ name_tokens_l                   ┃ name_tokens_r                   ┃ name_true_l                ┃ name_true_r                           ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ int64       │ int64       │ array<string>            │ array<string>            │ array<string>                     │ array<string>                     │ int64        │ int64        │ float64    │ float64    │ float64     │ float64     │ string                     │ string                                     │ string                        │ string                                              │ array<string>                   │ array<string>                   │ string                     │ string                                │
├─────────────┼─────────────┼──────────────────────────┼──────────────────────────┼───────────────────────────────────┼───────────────────────────────────┼──────────────┼──────────────┼────────────┼────────────┼─────────────┼─────────────┼────────────────────────────┼────────────────────────────────────────────┼───────────────────────────────┼─────────────────────────────────────────────────────┼─────────────────────────────────┼─────────────────────────────────┼────────────────────────────┼───────────────────────────────────────┤
│      170407 │      170408 │ ['B23B', 'F16L', ... +3] │ ['H02G']                 │ ['ADRIAAN', 'ADRIANUS', ... +48]  │ ['ADRIAAN', 'MARINUS', ... +1]    │       173039 │       173039 │       NULL │       NULL │        NULL │        NULL │ ABB                        │ ABB                                        │ ABB B.V.                      │ ABB B.V.,                                           │ ['ABB']                         │ ['ABB']                         │ ABB                        │ ABB V                                 │
│      317981 │      317982 │ ['A43B']                 │ ['A43B']                 │ ['ANDRU', 'ANGERS', ... +49]      │ ['ALLEN', 'GERD', ... +4]         │       319886 │       319886 │       NULL │  52.350000 │        NULL │    4.916667 │ ADIDAS INTERNATL MARKETING │ ADIDAS IUMLAUT OVER NTERNATIONAL MARKETING │ ADIDAS INTERNATL MARKETING BV │ adidas I{umlaut over (n)}ternational Marketing B.V. │ ['ADIDAS', 'INTERNATL', ... +1] │ ['ADIDAS', 'IUMLAUT', ... +3]   │ ADIDAS INTERNATL MARKETING │ ADIDAS IUMLAUT MARKETING NTERNAT OVER │
│      579028 │      579029 │ ['A61K', 'A61P', ... +2] │ ['A61K', 'A61P', ... +1] │ ['AVENTIS', 'DUCHAUSSOY', ... +9] │ ['BUMA', 'BURSI', ... +10]        │       569309 │       569309 │       NULL │       NULL │        NULL │        NULL │ A

The result of that was the two tables joined together, with a `_l` added
to all the columns from the left table, and a `_r` added to all the columns
from the right table.

By using this simple API, Blockers are easily composable and extendable.
For example, we can make an ensemble Blocker that takes multiple
other blockers, and returns the union of all their pairs.

In [7]:
from mismo.lib.geo import CoordinateBlocker

sub_blockers = [
    name_blocker,
    # Often, you want to look at pairs where records share some key.
    # There is KeyBlocker for this. This is equivalent to
    # `lambda left, right: left.coauthors == right.coauthors`
    block.KeyBlocker("coauthors", name="Coauthors Exact"),
    # You can also pass in deferreds for more complex logic.
    # And notice that the .unnest() actually changes the cardinality of the
    # key column, but the KeyBlocker can handle this.
    # Here, we are finding pairs where the arrays of classes have at least
    # one in common.
    block.KeyBlocker(_.classes.unnest(), name="Any Class"),
    # CoordinateBlocker is merely a class that implements a __call__ method,
    # which makes it a callable similar to the lambda above.
    # You can implement your own blockers by implementing __call__ for
    # any class. You could do complex logic in there, like
    # - Dropping any null records
    # - Using a UDF to computing the doublemetaphone of the name on the fly.
    # - Using a UDF to do an approximate-nearest-neighbor search in a vector database
    # - whatever you want! As long as it follows the required signature.
    CoordinateBlocker(
        distance_km=10, name="Coordinates Close", lat="latitude", lon="longitude"
    ),
]
# If you didn't care about the names of the rules, you could also just pass
# in the blockers directly, eg `rules=["classes", _.name_cleaned[0:3], ...]`.

featured = featured.cache()
# by passing in `labels=True`, an additional column `blocking_rules` is added
# to the output that shows which rules were used to generate each pair.
# This is slower, but later we want to see how many pairs each rule generated.
blocker = block.UnionBlocker(*sub_blockers, labels=True)
blocked = blocker(featured, featured)
blocked = blocked.cache()
blocked

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ record_id_l ┃ record_id_r ┃ blocking_rules ┃ classes_l                ┃ classes_r                 ┃ coauthors_l                       ┃ coauthors_r                         ┃ label_true_l ┃ label_true_r ┃ latitude_l ┃ latitude_r ┃ longitude_l ┃ longitude_r ┃ name_cleaned_l       ┃ name_cleaned_r     ┃ name_l                       ┃ name_r                   ┃ name_tokens_l               ┃ name_tokens_r             ┃ name_true_l          ┃ name_true_r          ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ int64       │ int64       │ array<string>  │ array<string>            │ array<string>             │ array<string>                     │ array<string>                       │ int64        │ int64        │ float64    │ float64    │ float64     │ float64     │ string               │ string             │ string                       │ string                   │ array<string>               │ array<string>             │ string               │ string               │
├─────────────┼─────────────┼────────────────┼──────────────────────────┼───────────────────────────┼───────────────────────────────────┼─────────────────────────────────────┼──────────────┼──────────────┼────────────┼────────────┼─────────────┼─────────────┼──────────────────────┼────────────────────┼──────────────────────────────┼──────────────────────────┼─────────────────────────────┼───────────────────────────┼──────────────────────┼──────────────────────┤
│        2909 │    35244259 │ ['Any Class']  │ ['A61B', 'A61N']         │ ['A01B', 'A01C', ... +98] │ ['DAVID', 'ELECTRONICS', ... +5]  │ ['ABDEL', 'ABRAHAN', ... +219]      │       402600 │     35272490 │       NULL │       NULL │        NULL │        NULL │ AGILENT TECHNOLOGIES │ UNILEVER           │ * AGILENT TECHNOLOGIES, INC. │ UNILEVER N.V.            │ ['AGILENT', 'TECHNOLOGIES'] │ ['UNILEVER']              │ AGILENT TECHNOLOGIES │ UNILEVER             │
│        3574 │    35244318 │ ['Any Class']  │ ['B01L', 'C11D', ... +3] │ ['A01H', 'A01N', ... +98] │ ['ALBERT', 'ALBERTO', ... +31]    │ ['ADAMS', 'ADRIAAN', ... +266]      │       569309 │     35272490 │       NULL │  51.916667 │        NULL │    4.500000 │ AKZO NOBEL           │ UNILEVER           │ * AKZO NOBEL N.V.            │ Unilever N.V.            │ ['AKZO', 'NOBEL']           │ ['UNILEVER']              │ AKZO NOBEL           │ UNILEVER             │
│        3575 │    35244259 │ ['Any Class']  │ ['B01F', 'C09K', ... +2] │ ['A01B', 'A01C', ... +98] │ ['CHRISTINA', 'ERNEST', ... +11]  │ ['ABDEL', 'ABRAHAN', ... +219]      │       569309 │     35272490 │       NULL │       NULL │        NULL │        NULL │ AKZO NOBEL           │ UNILEVER           │ * AKZO NOBEL NV              │ UNILEVER N.V.            │ ['AKZO', 'NOBEL']           │ ['UNILEVER']              │ AKZO NOBEL           │ UNILEVER             │
│        3779 │    35244318 │ ['Any Class']  │ ['G02B', 'G04G', ... +2] │ ['A01H', 'A01N', ... +98] │ ['EVAN', 'GRANT', ... +8]         │ ['ADAMS', 'ADRIAAN', ... +266]      │       656303 │     35272490 │      52.35 │  51.916667 │    4.916667 │    4.500000 │ ALCATEL     

In [8]:
blocked.blocking_rules.unnest().value_counts()

┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ blocking_rules    ┃ blocking_rules_count ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ string            │ int64                │
├───────────────────┼──────────────────────┤
│ Any Class         │               569034 │
│ Coauthors Exact   │                   98 │
│ Coordinates Close │               274831 │
│ Name First 3      │               215968 │
└───────────────────┴──────────────────────┘

By blocking, we reduced the number of needed pairs by a large factor.
In larger datasets, and with better blocking rules, this would be even more!

In [9]:
n_comparisons = blocked.count().execute()
n_naive = mismo.metrics.n_naive_comparisons(featured)
reduction_ratio = n_comparisons / n_naive
n_naive, n_comparisons, reduction_ratio

(2828631, 862446, 0.30489873016310715)

We can also inspect which blocking rules were most to blame for the generated
pairs. If some rules generate a huge amount of comparisons, it might be worth
trying to make them more restrictive so we get better performance. Or, if some
blocking rules aren't generating any comparisons, that might be an indication
that we have a bug in there somewhere.

In [10]:
mismo.block.upset_chart(blocked)

alt.VConcatChart(...)

OK, now that we have our candidate pairs generated, let's actually do the
comparing of pairs. There are many ways to do this, but one of the most common
is to generate a set of Comparison objects, each of which represents a
measurement of similarity based on some dimension (eg "location", "name", "price", etc).
Each Comparison is composed of Levels, which represent discrete levels of aggreement
(eg "exactly", "within 100km", "one or both values null", etc)

In [11]:
from mismo.compare import (
    LevelComparer,
    compare,
)
from mismo.lib.geo import distance_km
from mismo.sets import jaccard

name_comparison = LevelComparer(
    name="Name",
    levels=[
        ("exact", _.name_cleaned_l == _.name_cleaned_r),
        ("share 1 token", _.name_tokens_l.intersect(_.name_tokens_r).length() == 1),
        ("share 2+ tokens", _.name_tokens_l.intersect(_.name_tokens_r).length() >= 2),
    ],
)

classes_comparison = LevelComparer(
    name="Classes",
    levels=[
        ("exact", _.classes_l == _.classes_r),
        ("share 1 class", _.classes_l.intersect(_.classes_r).length() == 1),
        ("share 2+ classes", _.classes_l.intersect(_.classes_r).length() >= 2),
    ],
)

coauthors_comparison = LevelComparer(
    name="Coauthors",
    levels=[
        ("jaccard >= 0.5", jaccard(_.coauthors_l, _.coauthors_r) >= 0.5),
        ("jaccard >= 0.25", jaccard(_.coauthors_l, _.coauthors_r) >= 0.25),
        ("jaccard >= 0.1", jaccard(_.coauthors_l, _.coauthors_r) >= 0.1),
        ("jaccard >= 0.02", jaccard(_.coauthors_l, _.coauthors_r) >= 0.02),
    ],
)

coords_comparison = LevelComparer(
    name="Coords",
    levels=[
        ("exact", (_.latitude_l == _.latitude_r) & (_.longitude_l == _.longitude_r)),
        (
            "coords <= 10km",
            distance_km(
                lat1=_.latitude_l,
                lon1=_.longitude_l,
                lat2=_.latitude_r,
                lon2=_.longitude_r,
            )
            <= 10,
        ),
        (
            "coords <= 100km",
            distance_km(
                lat1=_.latitude_l,
                lon1=_.longitude_l,
                lat2=_.latitude_r,
                lon2=_.longitude_r,
            )
            <= 100,
        ),
        (
            "both coord missing",
            ibis.and_(
                _.latitude_l.isnull() | _.longitude_l.isnull(),
                _.latitude_r.isnull() | _.longitude_r.isnull(),
            ),
        ),
        (
            "one coord missing",
            ibis.or_(
                _.latitude_l.isnull() | _.longitude_l.isnull(),
                _.latitude_r.isnull() | _.longitude_r.isnull(),
            ),
        ),
    ],
)

comparisons = [
    name_comparison,
    classes_comparison,
    coauthors_comparison,
    coords_comparison,
]
compared = compare(blocked, *comparisons)
compared = compared.cache()
compared

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ record_id_l ┃ record_id_r ┃ blocking_rules ┃ classes_l                ┃ classes_r                 ┃ coauthors_l                       ┃ coauthors_r                         ┃ label_true_l ┃ label_true_r ┃ latitude_l ┃ latitude_r ┃ longitude_l ┃ longitude_r ┃ name_cleaned_l       ┃ name_cleaned_r     ┃ name_l                       ┃ name_r                   ┃ name_tokens_l               ┃ name_tokens_r             ┃ name_true_l          ┃ name_true_r          ┃ Name   ┃ Classes          ┃ Coauthors       ┃ Coords             ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ int64       │ int64       │ array<string>  │ array<string>            │ array<string>             │ array<string>                     │ array<string>                       │ int64        │ int64        │ float64    │ float64    │ float64     │ float64     │ string               │ string             │ string                       │ string                   │ array<string>               │ array<string>             │ string               │ string               │ string │ string           │ string          │ string             │
├─────────────┼─────────────┼────────────────┼──────────────────────────┼───────────────────────────┼───────────────────────────────────┼─────────────────────────────────────┼──────────────┼──────────────┼────────────┼────────────┼─────────────┼─────────────┼──────────────────────┼────────────────────┼──────────────────────────────┼──────────────────────────┼─────────────────────────────┼───────────────────────────┼──────────────────────┼──────────────────────┼────────┼──────────────────┼─────────────────┼────────────────────┤
│        2909 │    35244259 │ ['Any Class']  │ ['A61B', 'A61N']         │ ['A01B', 'A01C', ... +98] │ ['DAVID', 'ELECTRONICS', ... +5]  │ ['ABDEL', 'ABRAHAN', ... +219]      │       402600 │     35272490 │       NULL │       NULL │        NULL │        NULL │ AGILENT TECHNOLOGIES │ UNILEVER           │ * AGILENT TECHNOLOGIES, INC. │ UNILEVER N.V.            │ ['AGILENT', 'TECHNOLOGIES'] │ ['UNILEVER']              │ AGILENT TECHNOLOGIES │ UNILEVER             │ else   │ share 2+ classes │ else            │ both coord missing │
│        3574 │    35244318 │ ['Any Class']  │ ['B01L', 'C11D', ... +3] │ ['A01H', 'A01N', ... +98] │ ['ALBERT', 'ALBERTO', ... +31]    │ ['ADAMS', 'ADRIAAN', ... +266]      │       569309 │     35272490 │       NULL │  51.916667 │        NULL │    4.500000 │ AKZO NOBEL           │ UNILEVER           │ * AKZO NOBEL N.V.            │ Unilever N.V.            │ ['AKZO', 'NOBEL']           │ ['UNILEVER']              │ AKZO NOBEL           │ UNILEVER             │ else   │ share 2+ classes │ else            │ one coord missing  │
│        3575 │    35244259 │ ['Any Class']  │ ['B01F', 'C09K', ... +2] │ ['A01B', 'A01C', ... +98] │ ['CHRISTINA', 'ERNEST', ... +11]  │ ['ABDEL', 'ABRAHAN', ... +219]      │       569309 │     35272490 │       NULL │       NULL │        NULL │        NULL │ AKZO NOBE

The result above is the blocked table, with a column added for every `Comparison`.
The value of each column is the level that the record pair matched at.
For example, there is now a "Name" column,
filled with values like "exact", "share 1 token", etc.

Now that we have our features, we can use the Fellegi-Sunter model to train weights
for each of these features. This is a probabilistic model that is based on the concept
of odds. When you see an exact match on name, that increases the odds of a match
by some amount, maybe 10x. When you see a non-match on name, that decreases the odds
of a match by some amount, maybe 0.1x. We can either train this from labeled data,
or we can use unlabeled data using an algorithm called "Expectation Maximization".

In [12]:
weights = mismo.fs.train_using_em(comparisons, featured, featured, max_pairs=100_000)
# Can save and load weights
# weights.to_json("weights.json")
# weights = ComparisonWeights.from_json("weights.json")
weights.plot()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

alt.VConcatChart(...)

In the above plot, you can see the odds for "share 1 class" and "share 2+ classes"
are nearly the same. That doesn't quite make sense, and might be an indicator
that we should look more into this to see if there is a problem with the features
that we are using. But for now let's just move on.

Use the weights to score the record pairs, finding the odds for each
`Comparison``, and then combining them into an overall odds for the record pair.

In [13]:
scored = weights.score_compared(compared)
scored = scored.cache()
scored

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ record_id_l ┃ record_id_r ┃ blocking_rules ┃ classes_l                ┃ classes_r                 ┃ coauthors_l                       ┃ coauthors_r                         ┃ label_true_l ┃ label_true_r ┃ latitude_l ┃ latitude_r ┃ longitude_l ┃ longitude_r ┃ name_cleaned_l       ┃ name_cleaned_r     ┃ name_l                       ┃ name_r                   ┃ name_tokens_l               ┃ name_tokens_r             ┃ name_true_l          ┃ name_true_r          ┃ odds     ┃ Name   ┃ Name_odds ┃ Classes          ┃ Classes_odds ┃ Coauthors       ┃ Coauthors_odds ┃ Coords             ┃ Coords_odds ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ int64       │ int64       │ array<string>  │ array<string>            │ array<string>             │ array<string>                     │ array<string>                       │ int64        │ int64        │ float64    │ float64    │ float64     │ float64     │ string               │ string             │ string                       │ string                   │ array<string>               │ array<string>             │ string               │ string               │ float64  │ string │ float64   │ string           │ float64      │ string          │ float64        │ string             │ float64     │
├─────────────┼─────────────┼────────────────┼──────────────────────────┼───────────────────────────┼───────────────────────────────────┼─────────────────────────────────────┼──────────────┼──────────────┼────────────┼────────────┼─────────────┼─────────────┼──────────────────────┼────────────────────┼──────────────────────────────┼──────────────────────────┼─────────────────────────────┼───────────────────────────┼──────────────────────┼──────────────────────┼──────────┼────────┼───────────┼──────────────────┼──────────────┼─────────────────┼────────────────┼────────────────────┼─────────────┤
│        2909 │    35244259 │ ['Any Class']  │ ['A61B', 'A61N']         │ ['A01B', 'A01C', ... +98] │ ['DAVID', 'ELECTRONICS', ... +5]  │ ['ABDEL', 'ABRAHAN', ... +219]      │       402600 │     35272490 │       NULL │       NULL │        NULL │        NULL │ AGILENT TECHNOLOGIES │ UNILEVER           │ * AGILENT TECHNOLOGIES, INC. │ UNILEVER N.V.            │ ['AGILENT', 'TECHNOLOGIES'] │ ['UNILEVER']              │ AGILENT TECHNOLOGIES │ UNILEVER             │ 0.001210 │ else   │  0.000226 │ share 2+ classes │     6.577875 │ else            │       0.831345 │ both coord missing │    0.977997 │
│        3574 │    35244318 │ ['Any Class']  │ ['B01L', 'C11D', ... +3] │ ['A01H', 'A01N', ... +98] │ ['ALBERT', 'ALBERTO', ... +31]    │ ['ADAMS', 'ADRIAAN', ... +266]      │       569309 │     35272490 │       NULL │  51.916667 │        NULL │    4.500000 │ AKZO NOBEL           │ UNILEVER           │ * AKZO NOBEL N.V.            │ Unilever N.V.            │ ['AKZO', 'NOBEL']         

We can plot these compared pairs.
We can see which comparison levels are most common,
which occur together,
which lead to matches, and which lead to non-matches.

The exact match levels have the highest odds, and the
else levels have the lowest. The other levels are somewhere in between.

In [14]:
from mismo.compare import compared_dashboard

compared_dashboard(compared, comparisons=comparisons, weights=weights)

It looks like an odds of 50 seems to separate the pairs between non-matches
and matches.
If I hover over the above chart, I can see that pretty much all the "else" comparisons
are in the low cluster, and all the "exact" comparisons are in the high cluster.

In [15]:
odds_threshold = 50
(scored.odds >= odds_threshold).value_counts()

┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ GreaterEqual(odds, 50) ┃ GreaterEqual(odds, 50)_count ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ boolean                │ int64                        │
├────────────────────────┼──────────────────────────────┤
│ True                   │                       129299 │
│ False                  │                       733147 │
└────────────────────────┴──────────────────────────────┘

Let's be really picky and only take the most likely matches as true matches, and
then perform connected components to label each patent with its inventor:

In [16]:
links = scored[_.odds >= odds_threshold]
links = links.cache()
print(links.count().execute())
labels = mismo.cluster.connected_components(links, nodes=featured.record_id)
print(labels.count().execute())
labels

129299
2379


┏━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ record_id ┃ component ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━┩
│ int64     │ int64     │
├───────────┼───────────┤
│      3780 │      2909 │
│    578450 │      2909 │
│    578572 │      2909 │
│    578570 │      2909 │
│   7651387 │      2909 │
│    578553 │      2909 │
│    578625 │      2909 │
│    578546 │      2909 │
│  32621577 │      2909 │
│  32621582 │      2909 │
│         … │         … │
└───────────┴───────────┘

In [17]:
patents_labeled = patents.left_join(labels, "record_id")
patents_labeled = patents_labeled.relocate("component", after="label_true")
patents_labeled.order_by("component").head(20)

┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ record_id ┃ label_true ┃ component ┃ name_true            ┃ name                         ┃ latitude ┃ longitude ┃ coauthors                                                                        ┃ classes                                                          ┃ record_id_right ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ int64     │ int64      │ int64     │ string               │ string                       │ float64  │ float64   │ string                                                                           │ string                                                           │ int64           │
├───────────┼────────────┼───────────┼──────────────────────┼──────────────────────────────┼──────────┼───────────┼──────────────────────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────────┼─────────────────┤
│      2909 │     402600 │      2909 │ AGILENT TECHNOLOGIES │ * AGILENT TECHNOLOGIES, INC. │     0.00 │  0.000000 │ KONINK PHILIPS ELECTRONICS N V**DAVID E  SNYDER**THOMAS D  LYSTER                │ A61N**A61B                                                       │            2909 │
│      3574 │     569309 │      2909 │ AKZO NOBEL           │ * AKZO NOBEL N.V.            │     0.00 │  0.000000 │ TSJERK  HOEKSTRA**ANDRESS K  JOHNSON**TERESA MARIE  CHERON**ALBERTO  SLIKTA**JA… │ G01N**B01L**C11D**G02F**F16L                                     │            3574 │
│      3575 │     569309 │      2909 │ AKZO NOBEL           │ * AKZO NOBEL NV              │     0.00 │  0.000000 │ WILLIAM JOHN ERNEST  PARR**HANS  OSKARSSON**MARTIN  HELLSTEN**KORNELIS  OVERKEM… │ C09K**F17D**B01F**C23F                                           │            3575 │
│      3779 │     656303 │      2909 │ ALCATEL              │ * ALCATEL N.V.               │    52.35 │  4.916667 │ GUENTER  KOCHSMEIER**ZBIGNIEW  WIEGOLASKI**EVAN JOHN  STANBURY**PETER GRANT  JE… │ G02B**G04G**H02G**G06F                                           │            3779 │
│      3780 │     656303 │      2909 │ ALCATEL              │ * ALCATEL N.V.               │    52.35 │  4.916667 │ ZILAN  MANFRED**JOSIANE  RAMOS**DUANE LYNN  MORTENSEN**CHRISTIAN  LE SERGENT     │ H03G**B05D**H04L**H04B**C03B**C03C**G02B**H01B                   │            3780 │
│      3782 │     656303 │      2909 │ ALCATEL              │ * ALCATEL N.V.               │     0.00 │  0.000000 │ OLIVIER  AUDOUIN**MICHEL  SOTOM**JEAN MICHEL  GABRIAGUES                         │ H04B**H01S**H04J                                                 │            3782 │
│     15041 │    4333661 │      2909 │ CANON EUROPA         │ * CANON EUROPA N.V           │     0.00 │  0.000000 │ LEE  RICKLER**SIMON  PARKER**CANON RES CENT EURO **RAKEFET  SAGMAN**TIMOTHY FRA… │ G06F                                                             │           15041 │
│     15042 │    4333661 │      2909 │ CANON EUROPA         │ * CANON EUROPA N.V.          │     0.00 │  0.000000 │ QI HE  HONG**ADAM MICHAEL  BAUMBERG**ALEXANDER RALPH  LYONS                      │ G06T**G01B                                                       │           15042 │
│     15043 │    4333661 │      2909 │ CANON EUROPA         │ * CANON EUROPA NV            │     0.00 │  0.000000 │ NILESH  PATHAK**MASAMICHI  MASUDA** CANON TECHNOLOGY EURO **PATRICK WILLIAM  MO… │ H04B**G06T**G06F**H04M**H04N**H04Q**G03B**B41J**G01B**G06Q       │           15043 │
│     25387 │    7650783

Now let's evaluate how good our labeling is. Mismo wraps all of the evaluation
metrics from sklearn, so we can use them with Ibis Tables.

In [18]:
labels_true = patents_labeled.select("record_id", label=_.label_true)
labels_pred = patents_labeled.select("record_id", label=_.component)
print(mismo.metrics.adjusted_rand_score(labels_true, labels_pred))
print(mismo.metrics.homogeneity_score(labels_true, labels_pred))
print(mismo.metrics.completeness_score(labels_true, labels_pred))

0.10221328294557343
0.32421809375569566
0.9053393388568964
